In [200]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel
from dotenv import load_dotenv, find_dotenv
from langchain_together import ChatTogether
from pydantic import BaseModel, Field
import smtplib
from email.message import EmailMessage
import ssl
from twilio.rest import Client
from twilio.twiml.voice_response import VoiceResponse
from decouple import config

In [201]:
load_dotenv(find_dotenv())

True

In [192]:
def make_phone_call(recipient_phone, call_script, twilio_number, account_sid, auth_token):
    """
    Makes a phone call to the recipient and speaks the script using Twilio.
    
    Parameters:
        recipent_phone (str): Recipient's phone number in E.164 format (e.g., "+1234567890").
        call_script (str): The text to be spoken during the call.
        twilio_number (str): Your Twilio phone number (E.164 format).
        account_sid (str): Twilio account SID.
        auth_token (str): Twilio authentication token.
    """
    # Initialize Twilio client
    client = Client(account_sid, auth_token)
    
    # Create TwiML (Telephony Markup Language) to define the call behavior
    twiml_response = VoiceResponse()
    twiml_response.say(call_script, voice='alice')  # 'alice' is a natural-sounding voice
    
    try:
        # Initiate the call
        call = client.calls.create(
            twiml=twiml_response.to_xml(),
            from_=twilio_number,
            to=recipient_phone
        )
        print(f"Call initiated! Call SID: {call.sid}")
    except Exception as e:
        print(f"Failed to make call: {e}")

In [223]:
ACCOUNT_SID = config("ACCOUNT_SID")
AUTH_TOKEN = config("AUTH_TOKEN")
TWILIO_NUMBER = config("TWILIO_NUMBER")
RECIPIENT_NUMBER = "+2348101116037"
EMAIL_PASSWORD = config("EMAIL_PASSWORD")

In [194]:
# Call the function
make_phone_call(
    recipient_phone=RECIPIENT_NUMBER,
    call_script="Hello! This is an automated call. Thank you for testing our service.",
    twilio_number=TWILIO_NUMBER,
    account_sid=ACCOUNT_SID,
    auth_token=AUTH_TOKEN
)

Call initiated! Call SID: CA88620e6ab02cf59f9af36e95dcb81e0b


In [57]:
def send_email(subject, recipient, body, sender_email, sender_password):
    """
    Sends an email using the provided details and SMTP server (e.g., Gmail).
    
    Parameters:
    - subject (str): Email subject
    - recipient (str): Recipient email address
    - body (str): Email body content
    - sender_email (str): Sender's email address (requires SMTP access)
    - sender_password (str): Sender's email password or app-specific password
    """
    # Create the email message
    msg = EmailMessage()
    msg.set_content(body)
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = recipient

    # Set up SSL context and SMTP server
    context = ssl.create_default_context()
    
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as server:
            server.login(sender_email, sender_password)
            server.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

In [202]:
send_email(
    subject="Hello from Python!",
    recipient="paulobiahu@gmail.com",
    body="This is a test email sent using Python.",
    sender_email="paulobiahu@gmail.com",
    sender_password=EMAIL_PASSWORD
)

Email sent successfully!


In [224]:
def read_file(file_path):
    """
    Reads a JSON, Excel, CSV, Parquet, Feather, or TSV file into a Pandas DataFrame.
    
    Parameters:
        file_path (str): Path to the file.
    
    Returns:
        pd.DataFrame: DataFrame containing the file data.
    """
    try:
        if file_path.endswith(".csv"):
            return pd.read_csv(file_path)
        elif file_path.endswith(".json"):
            return pd.read_json(file_path)
        elif file_path.endswith(".xlsx") or file_path.endswith(".xls"):
            return pd.read_excel(file_path)
        elif file_path.endswith(".parquet"):
            return pd.read_parquet(file_path)
        elif file_path.endswith(".feather"):
            return pd.read_feather(file_path)
        elif file_path.endswith(".tsv"):
            return pd.read_csv(file_path, sep="\t")
        else:
            raise ValueError("Unsupported file format. Please use JSON, Excel, CSV, Parquet, Feather, or TSV.")
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

In [225]:
def get_company_info(file_path):
    """
    Extracts company-related information from the given file.
    
    Parameters:
        file_path (str): Path to the file.
    
    Returns:
        tuple: Data for Company Names, Fields, Email, Phone, Preferred Channel, Objections, Engagement Level, and Notes.
    """
    df = read_file(file_path)
    if df is None:
        return None
    
    required_columns = [
        "Company_Names", "Company_Fields", "Email_Address", "Phone_Number", 
        "Preferred_Channel", "Objections", "Engagement_Level", "Company_Notes"
    ]
    
    missing_cols = [col for col in required_columns if col not in df.columns]
    
    if missing_cols:
        print(f"Error: Missing columns {missing_cols}.")
        print("Ensure all required columns are present in the file.")
        return None
    
    return (
        df["Company_Names"].values, df["Company_Fields"].values, df["Email_Address"].values, df["Phone_Number"].values, 
        df["Preferred_Channel"].values, df["Objections"].values, df["Engagement_Level"].values, df["Company_Notes"].values
    )

In [226]:
get_company_info('names_and_fields.csv')

(array(['Tech Corp', 'Green Energy', 'MediCare', 'EduWorld'], dtype=object),
 array(['Software', 'Renewable Energy', 'Healthcare', 'Education'],
       dtype=object),
 array(['contact@techcorp.com', 'info@greenenergy.com',
        'support@medicare.com', 'hello@eduworld.com'], dtype=object),
 array(['123-456-7890', '987-654-3210', '555-666-7777', '444-333-2222'],
       dtype=object),
 array(['email', 'phone', 'email', 'phone'], dtype=object),
 array([nan, 'Price concerns', nan, 'Service issues'], dtype=object),
 array(['High', 'Medium', 'Low', 'High'], dtype=object),
 array(['Leading software company', 'Expanding solar solutions',
        'Well-known in medical field', 'Provides online courses'],
       dtype=object))

In [227]:
llm = ChatTogether(
    model="meta-llama/Llama-3-70b-chat-hf",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [228]:
class EmailAdvise(BaseModel):
    recipient_email: str = Field(description="The **email address of the recipient** (as provided in the input).")
    recipient_phone: str = Field(description="The **phone number of the recipient** (as provided in the input)")
    subject: str = Field(description="A compelling **email subject line** that grabs attention and encourages the recipient to open the email. ")
    email: str = Field(description="A cold email personalized based on the industry, engagement level, and objections.")
    call_script: str = Field(description="A structured cold call script for sales representatives to use.")
    advise: str = Field(description="A strategy to further engage the client based on their industry and previous response.")

In [229]:
parser = JsonOutputParser(pydantic_object=EmailAdvise)

In [230]:
prompt = """
You are an AI-powered Cold Outreach Assistant for an insurance company specializing in personalized marketing. Your goal is to craft **highly engaging cold emails** and **concise, high-converting cold call scripts** tailored to prospects' industries, prior engagement levels, and potential objections. Additionally, you will provide strategic follow-up advice to further engage the client.

---

### **Task**  
Given the following details, generate a JSON response containing:  

1. `recipient_email`: The **email address of the recipient** (as provided in the input).  
2. `recipient_phone`: The **phone number of the recipient** (as provided in the input).  
3. `subject`: A compelling **email subject line** that grabs attention and encourages the recipient to open the email.  
4. `email`: A **personalized cold email** that follows this structure:  
   - **[Opening sentence]**: Connect with their industry, a recent trend, or challenge.  
   - **[Value proposition]**: How your insurance solution helps companies in their industry.  
   - **[Objection handling]**: Address a common concern relevant to their industry.  
   - **[CTA]**: Suggest a **quick call, demo, or free consultation**.  

5. `call_script`: A **short but effective cold call script** (under 100 words) that:  
   - Starts with a **brief introduction**.  
   - Highlights a **pain point** relevant to their industry.  
   - Presents the **key benefit of the insurance** in a single sentence.  
   - Handles a **likely objection** in one sentence.  
   - Ends with a **clear CTA** (e.g., booking a call, requesting more info).  

6. `advise`: A **follow-up strategy** to keep the client engaged based on their industry and potential objection.  

---

### **Input**  
- **Recipient Email**: {recipient_email}  
- **Recipient Phone**: {recipient_phone}  
- **Company Name**: {company_name}  
- **Industry**: {industry}  
- **Engagement Level**: {engagement_level}  
- **Potential Objection**: {objection}  
- **Insurance Company Name**: {insurance_company_name}  
- **Sender Name**: {sender_name}  

---

###Note: The email is sent to {company_name} so the greeting at the beginning of the email should be attributed to {company_name}

---

### **Output Format (Strict JSON)**  
{{
  "recipient_email": "{recipient_email}",
  "recipient_phone": "{recipient_phone}",
  "subject": "[Compelling subject line]",
  "email": "[Generated cold email]",
  "call_script": "[Short, high-impact cold call script]",
  "advise": "[Follow-up strategy and recommendations]"
}}
"""  


In [231]:
prompt_template = PromptTemplate(
    template=prompt,
    input_variables=["company_name", "industry", "engagement_level", "objection"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [232]:
first_chain = prompt_template | llm | parser

In [233]:
response = first_chain.invoke({
    "company_name": "Moniepoint",
    "industry": "Finance",
    "engagement_level": "low",
    "objection": "Security, We’re already insured, Trust with document issues",
    "insurance_company_name": "DigitalInsure",
    "sender_name": "Paul Obiahu",
    "recipient_email": "paulobiahu2@gmail.com",
    "recipient_phone": "+2348101116037"
})

In [234]:
print(response)

{'recipient_email': 'paulobiahu2@gmail.com', 'recipient_phone': '+2348101116037', 'subject': "Protecting Moniepoint's Financial Future with DigitalInsure", 'email': "Hi Moniepoint Team,\n\nAs a leading player in the finance industry, you understand the importance of mitigating risks and ensuring business continuity. With the increasing threat of cyber attacks and data breaches, it's crucial to have a robust insurance plan in place.\n\nAt DigitalInsure, we specialize in providing tailored insurance solutions to finance companies like yours. Our policies are designed to protect your business from unforeseen events, ensuring you can focus on growth and innovation.\n\nI understand that security concerns may be top of mind, especially when it comes to document management. Rest assured that our insurance solutions prioritize data protection and confidentiality, giving you peace of mind.\n\nWould you be open to a quick call to discuss how DigitalInsure can help safeguard Moniepoint's financia

In [235]:
print(response['recipient_phone'])

+2348101116037


In [236]:
print(response['recipient_email'])

paulobiahu2@gmail.com


In [237]:
print(response['subject'])

Protecting Moniepoint's Financial Future with DigitalInsure


In [238]:
print(response['call_script'])

Hi, this is Paul Obiahu from DigitalInsure. I'm calling because I understand the finance industry is prone to cyber attacks, and I'd like to discuss how our insurance solutions can help protect Moniepoint from these risks. Our policies prioritize data security and confidentiality, addressing concerns around document management. Are you currently satisfied with your insurance coverage? Would you be open to exploring alternative options that better suit your needs? Let's schedule a quick call to discuss further.


In [239]:
print(response['advise'])

Given Moniepoint's low engagement level, it's essential to prioritize building trust and establishing credibility. Consider sending a follow-up email with a case study or success story from a similar finance company that has benefited from DigitalInsure's insurance solutions. This will help address potential objections around security and trust. Additionally, offer a free consultation or demo to provide value upfront and showcase the expertise of DigitalInsure.


In [240]:
print(response['email'])

Hi Moniepoint Team,

As a leading player in the finance industry, you understand the importance of mitigating risks and ensuring business continuity. With the increasing threat of cyber attacks and data breaches, it's crucial to have a robust insurance plan in place.

At DigitalInsure, we specialize in providing tailored insurance solutions to finance companies like yours. Our policies are designed to protect your business from unforeseen events, ensuring you can focus on growth and innovation.

I understand that security concerns may be top of mind, especially when it comes to document management. Rest assured that our insurance solutions prioritize data protection and confidentiality, giving you peace of mind.

Would you be open to a quick call to discuss how DigitalInsure can help safeguard Moniepoint's financial future? I'd be happy to schedule a demo or free consultation at your convenience.

Best regards,
Paul Obiahu


In [241]:
def send_cold_email(generated_inputs):
    try:
        send_email(
            subject=generated_inputs['subject'],
            recipient=generated_inputs['recipient_email'],
            body=generated_inputs['email'],
            sender_email="paulobiahu@gmail.com",
            sender_password=EMAIL_PASSWORD
        )
        return "Success"
    except Exception as e:
        return "Failure"

In [242]:
def send_cold_call(generated_inputs):
    try:
        make_phone_call(
            recipient_phone=generated_inputs['recipient_phone'],
            call_script=generated_inputs['call_script'],
            twilio_number=TWILIO_NUMBER,
            account_sid=ACCOUNT_SID,
            auth_token=AUTH_TOKEN
        )
        return "Success"
    except Exception as e:
        return "Failure"

In [243]:
def get_advise(generated_inputs):
    return generated_inputs['advise']

In [244]:
cold_email_runnable = RunnableLambda(send_cold_email)
cold_call_runnable = RunnableLambda(send_cold_call)
get_advise_runnable = RunnableLambda(get_advise)

In [245]:
email_and_call_chain = RunnableParallel(
    email_status=cold_email_runnable,  
    call_status=cold_call_runnable,  
    advise_status=get_advise_runnable  
)

full_chain = first_chain | email_and_call_chain

In [249]:
response = full_chain.invoke({
    "company_name": "Moniepoint",
    "industry": "Finance",
    "engagement_level": "high",
    "objection": "Security, We’re already insured, Trust with document issues",
    "insurance_company_name": "DigitalInsure",
    "sender_name": "Paul Obiahu",
    "recipient_email": "paulobiahu3@gmail.com",
     "recipient_phone": "+2348101116037"
})

Call initiated! Call SID: CAdf70f4042ff0666dae9b02aec9cf8eb2
Email sent successfully!


In [250]:
response

{'email_status': 'Success',
 'call_status': 'Success',
 'advise_status': "Given Moniepoint's high engagement level, I recommend a follow-up email or call within 3-5 days to reiterate the value proposition and address any potential concerns. Since security is a top concern, consider sharing a case study or testimonial from a similar finance company that has benefited from DigitalInsure's insurance solutions. This will help build trust and credibility, increasing the likelihood of conversion."}